In [1]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time 
import os
import json
import urllib


In [2]:
# Some urls are blocked from untrusted sites, the below code make python environment to trust every site
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def download_images():
    
    f = open('site_urls.txt','r')
    search_url = f.readlines()[30]
    print(search_url)
   
    # path of chorme drive to run selenium
    path = r'/Users/priyankasurapaneni/Documents/RA/chromedriver'
    
    driver = webdriver.Chrome(path)
    
    k = open('wildlife keywords.txt','r')
    kw = k.readlines()
    
    for i in range(0,len(kw)):
        print(i)
        keyW = kw[i]
        
        driver.get(search_url)

        time.sleep(2)
        
        # search text box 
        box = driver.find_element_by_xpath('//*[@id="header-search"]/input[2]')  
        box.clear()
        #sending the keyword
        box.send_keys(keyW)
        time.sleep(2)
        
        # try block if there are no ad results for the keyword it will continue with next keyword
        try:
            # collecting links of ad results
            links = driver.find_element_by_xpath('//*[@id="product-list"]')
            sub_lnks = links.find_elements_by_tag_name('a')

            lnks_lst = []

            for lnk in sub_lnks:
                lnks_lst.append(lnk.get_attribute('href'))
            print(lnks_lst[0:3])

            # making a directory like URL+Keyword_number , example 30_0 , 30_1,...
            if lnks_lst:
                if not os.path.exists(str(30)+'_'+str(i)):
                    os.mkdir(str(30)+'_'+str(i))
                    
            #  iterating through ads results 
            for j in range(1,3):
                driver.get(lnks_lst[j])
                print(lnks_lst[j])
                time.sleep(2)
                # Title capturing
                sub_txt = driver.find_element_by_xpath('//*[@id="product-title"]/h1').text
                
                #making directory inside the above created directory like 0 means first ad, 1 for second ad....
                if not os.path.exists(str(30)+'_'+str(i) +'/'+ str(j)):
                    os.mkdir(str(30)+'_'+str(i) +'/'+ str(j))
                # making dictionary
                details = {
                    'name' : sub_txt,
                    'price' : driver.find_element_by_xpath('//*[@id="price"]').text,
                    'location': driver.find_element_by_xpath('//*[@id="seller-location"]/ul/li').text,
                    'details':None,
                    'description' : driver.find_element_by_xpath('//*[@id="expandedDetailsContent"]').get_attribute("innerText"),
                    'url' : lnks_lst[j]
                }
                
                # writing the dictionary to text file like 0.txt for first ad ....
                with open(str(30)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '.txt', 'w') as tfile:
                     tfile.write(json.dumps(details))
                
                # collecting image urls
                image_lst = []
                elem1 = driver.find_element_by_xpath('//*[@id="product-image-container"]')
                sub = elem1.find_elements_by_tag_name('img')
                for k,l in enumerate(sub):
                    src = l.get_attribute('src')
                    image_lst.append(src)
                    print(src)
                    # collecting images and saving it inside the above created directory like 0_0.jpg for first ad first image
                    # 0_1.jpg for first ad second image ...
                    if src != None:
                        urllib.request.urlretrieve(str(src), os.path.join(str(30)+'_'+str(i), str(j), str(j)+'_'+str(k)+'.jpg'))
                # writing image urls to text file like 0_imgs.txt for first ad ....
                with open(str(30)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '_imgs.txt', 'w') as tfile:
                     tfile.write(json.dumps(image_lst)) 
        except:
            continue
            
    driver.close()
    

In [4]:
def main():
    download_images()
    
if __name__ == "__main__":
    main()

http://www.ecrater.com/filter.php?a=1&keywords=[KW]&view=list&sort=price_desc

0
['https://www.ecrater.com/p/40747526/plant-30-mature-with-flowers-aranto-alligator?keywords=alligator', 'https://www.ecrater.com/p/40747526/plant-30-mature-with-flowers-aranto-alligator?keywords=alligator', 'http://enternity.ecrater.com/']
https://www.ecrater.com/p/40747526/plant-30-mature-with-flowers-aranto-alligator?keywords=alligator
https://s.ecrater.com/stores/545250/620c465190e35_545250b.jpg
http://enternity.ecrater.com/
1
['https://www.ecrater.com/p/19649351/gelpi-retractr-volkman-bone-curette?keywords=alligator%2Bbone', 'https://www.ecrater.com/p/19649351/gelpi-retractr-volkman-bone-curette?keywords=alligator%2Bbone', 'http://oneclass.ecrater.com/']
https://www.ecrater.com/p/19649351/gelpi-retractr-volkman-bone-curette?keywords=alligator%2Bbone
https://s.ecrater.com/stores/275853/533278c4a1996_275853b.jpg
https://s.ecrater.com/stores/275853/533278c4a1996_275853s.jpg
https://s.ecrater.com/stores/27